# Upload datasets to elastisearch

#### Bringing mapped baseline and movie scripts into a valid json format and push them to elasticsearch running on server.

1. Map baseline to movie scripts

In [126]:
%%bash
. ~/.bashrc
python3 -m pip install -r requirements.txt
# Install necessary packages

bash: line 1: /Users/leonremke/.bashrc: No such file or directory


In [127]:
# index key for further queries
index = "raw_movies"
# Elasticsearch credentials
user = ""
pw = ""
host = "https://app.leon-remke.jakob-hennighausen.melkonyan-davit.de/api/"

In [167]:
# Imorts
import os

from elasticsearch7 import Elasticsearch
import numpy as np
import spacy
import matplotlib
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from collections import Counter

In [168]:

es = Elasticsearch(
    hosts=host,
    http_auth=(user, pw)
)

es.info()

{'name': 'ubuntu-elastic-search',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'C3IT-gleTX-yJIPo-o8IxA',
 'version': {'number': '7.17.8',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '120eabe1c8a0cb2ae87cffc109a5b65d213e9df1',
  'build_date': '2022-12-02T17:33:09.727072865Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [169]:
pg_mapping = np.genfromtxt('./baseline/imdb_id_with_age_rating_list.txt', delimiter=',', dtype=str)

In [170]:
def conv_nan(row):
    if row[1] == 'no age rating found':
        row[1] = 'None'
    return row

In [171]:
# First apply the conv() filtering on the word count
pg_mapping = np.array([conv_nan(row) for row in pg_mapping])
pg_mapping

array([['tt0032138', 'None'],
       ['tt0035423', 'PG-13'],
       ['tt0038650', 'PG'],
       ...,
       ['tt7160070', 'NR'],
       ['tt7180392', 'None'],
       ['tt7672188', 'None']], dtype='<U19')

In [172]:
def get_string_from_file(file):
    with open('./script/' + file, 'r') as f:
        return f.read()

In [173]:
from os import listdir
from os.path import isfile, join

def get_files_from_dir(dir):
    onlyfiles = [f for f in listdir(dir) if isfile(join(dir, f))]
    return onlyfiles

In [174]:
output_file = open("elastic_search_init.json", "w")

for idx, file in enumerate(get_files_from_dir('./script')):
    script = get_string_from_file(file)
    script = script.replace("'", " ").replace('"', ' ').replace('\n', ' ').replace('\r', ' ')
    imdb_id = file.split('.')[0]
    pg_rating = pg_mapping[np.where(pg_mapping[:,0] == imdb_id)][0][1]
    value_str = '{"imdb_id": "' + imdb_id + '", "movie_script": "' + script + '", "age_rating": "' + str(pg_rating) + '"}'
    output_file.write('{"index":{"_index": "' + str(index) + '", "_id": "' + str(idx + 1) + '"}}' + '\n')
    output_file.write(value_str + '\n')
    if idx == 1:
        break
output_file.close()
    # Not working :(((
    # es.index(index='movies', document={"imdb_id": file, "script": script, "age_rating": pg_rating})
    # es.index(index=index, document={"imdb_id": file, "script": script, "age_rating": pg_rating})

In [175]:
%%bash
. ~/.bashrc
curl -u elasticuser:blaubarsch5 -H "Content-Type:application/x-ndjson" -XPOST https://app.leon-remke.jakob-hennighausen.melkonyan-davit.de/api/_bulk?pretty --data-binary "@elastic_search_init.json"



bash: line 1: /Users/leonremke/.bashrc: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  462k  100   770  100  462k    189   113k  0:00:04  0:00:04 --:--:--  114k


{
  "took" : 59,
  "errors" : false,
  "items" : [
    {
      "index" : {
        "_index" : "raw_movies",
        "_type" : "_doc",
        "_id" : "1",
        "_version" : 4,
        "result" : "updated",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 349,
        "_primary_term" : 1,
        "status" : 200
      }
    },
    {
      "index" : {
        "_index" : "raw_movies",
        "_type" : "_doc",
        "_id" : "2",
        "_version" : 3,
        "result" : "updated",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 350,
        "_primary_term" : 1,
        "status" : 200
      }
    }
  ]
}


In [163]:
es.search(index=index, query={
        "match_all": {}
     })

/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/elasticsearch7/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/elasticsearch7/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in search requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/elasticsearch7/connection/base.py:200: ElasticsearchWarning: this request accesses system indices: [.apm-agent-configuration, .apm-custom-link, .kibana_7.17.7_001, .kibana_7.17.8_001, .kibana_task_manager_7.17.7_001, .kibana_task_manager_7.17.8

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 9, 'successful': 9, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [166]:
mapping = es.indices.get_mapping(index=index)
mapping

/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/elasticsearch7/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


RequestError: RequestError(400, 'illegal_argument_exception', 'Types cannot be provided in get mapping requests, unless include_type_name is set to true.')

In [139]:
for idx, file in enumerate(get_files_from_dir('./script')):
    script = get_string_from_file(file)
    imdb_id = file.split('.')[0]
    pg_rating = pg_mapping[np.where(pg_mapping[:,0] == imdb_id)][0][1]
    # Not working :(((
    es.index(index=index, doc_type=index, document={"imdb_id": file, "script": script, "age_rating": pg_rating})
    # es.index(index=index, document={"imdb_id": file, "script": script, "age_rating": pg_rating})

/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/elasticsearch7/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/elasticsearch7/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


TransportError: TransportError(500, 'string_index_out_of_bounds_exception', 'Index 0 out of bounds for length 0')